In [3]:
import cvxpy as cp
import cvxpylayers
import matplotlib.pyplot as plt
import numpy as np
import torch
from algorithms import fit
from cvxpylayers.torch import CvxpyLayer
cp.__version__, cvxpylayers.__version__

('1.1.1', '0.1.4')

In [10]:
m, n  = 4, 3
x = cp.Variable(m, nonneg=True)
c = cp.Parameter(m, nonneg=True)
A = cp.Parameter((n,m))
b = cp.Parameter(n)
c_val = np.array([1,3,1,1])
b_val = np.array([2, -2, 0])
A_val = np.array([[1, 1, 0, -1],
     [0, -1, -1, 1],
     [1, 0, -1, 0]])
objective = cp.sum(cp.multiply(c,x))
constraints = [(cp.matmul(A,x) == b)]
prob = cp.Problem(cp.Minimize(objective), constraints)
c.value = c_val
A.value = A_val
b.value = b_val
assert prob.is_dpp()


# prob.solve()
layer = CvxpyLayer(prob, parameters=[c, A, b], variables=[x])
A_tf = torch.tensor(A.value)
c_tf = torch.tensor(c.value)
b_tf = torch.tensor(b.value)
solution, = layer(c_tf, A_tf, b_tf)
solution

tensor([2, 0, 2, 0])

In [19]:
# Define convex optimization model
m = 2

y = cp.Variable(m)
units = cp.Variable(m)
t = cp.Variable(m)
alpha = cp.Parameter(m, nonneg=True)
inverse_p = cp.Parameter(m, nonneg=True)
B = cp.Parameter(nonneg=True)

objective = cp.sum(t)
constraints = [cp.sum(y) == B, y >= 0,
               -cp.exp(-cp.multiply(alpha, units)) >= cp.multiply(alpha, t),
               units == cp.multiply(y, inverse_p)]
prob = cp.Problem(cp.Maximize(objective), constraints)

layer = CvxpyLayer(prob, [B, inverse_p, alpha], [y])

In [20]:
# Get data
def get_data(N, m, alpha):
    P = torch.rand(N, m)
    B = torch.rand(N) * 10
    Y = layer(B, 1 / P, alpha)[0]
    Y = Y * (torch.rand_like(Y) + .5)
    print(Y)
    Y = Y * B[:,None] / Y.sum(1)[:,None]
    print(Y)
    return torch.cat([B.unsqueeze(1), P], axis=1), Y

torch.manual_seed(0)
alpha_true = torch.rand(m)
X, Y = get_data(100, m, alpha_true)
Xval, Yval = get_data(50, m, alpha_true)
Y[:3]

tensor([[ 1.5143e+00,  2.6325e+00],
        [ 1.6824e+00,  5.6986e+00],
        [ 1.3005e+00,  3.5489e+00],
        [ 2.1518e+00,  2.8119e+00],
        [ 1.5775e+00,  2.2203e+00],
        [ 8.8233e-01,  1.0705e+00],
        [ 3.4428e+00,  5.1719e+00],
        [ 1.6413e+00,  1.6674e+00],
        [ 8.0738e-01,  1.8631e+00],
        [ 6.6262e-01,  1.1376e+00],
        [ 1.3238e+00,  9.8398e+00],
        [ 1.0686e+00,  1.3730e+00],
        [ 1.0213e+00,  1.9911e+00],
        [ 9.8070e-01,  4.3659e-01],
        [ 2.7003e+00,  3.0950e+00],
        [ 1.4695e+00,  1.6646e+00],
        [ 3.3607e+00,  3.8374e+00],
        [ 3.6145e+00,  5.7097e+00],
        [ 6.3486e-01,  1.0484e+00],
        [ 1.4784e+00,  4.4465e-01],
        [ 1.3928e+00,  9.1518e-01],
        [ 2.6453e+00,  2.4506e+00],
        [ 1.6617e+00,  2.3505e-01],
        [ 1.7866e+00,  2.5653e+00],
        [ 4.0283e+00,  3.0839e+00],
        [ 6.9896e-01,  2.9140e+00],
        [ 6.3535e-01,  1.0669e+00],
        [ 2.9864e+00,  7.017

tensor([[2.3216, 4.0360],
        [1.8880, 6.3953],
        [1.4536, 3.9668]])

In [4]:
loss_fn = torch.nn.MSELoss()

In [5]:
torch.manual_seed(1)
alpha = torch.rand(m)
alpha.requires_grad_(True)
def loss(X, Y, alpha):
    Yhat = layer(X[:,0], 1. / X[:,1:], alpha)[0]
    return loss_fn(Y, Yhat)

def callback():
    alpha.data = torch.max(alpha.data, torch.zeros_like(alpha.data))

print(loss(X, Y, alpha_true), loss(Xval, Yval, alpha_true))
print(loss(X, Y, alpha), loss(Xval, Yval, alpha))

val_losses, train_losses = fit(lambda X, Y: loss(X, Y, alpha), [alpha], X, Y, Xval, Yval,
                               opt=torch.optim.Adam, opt_kwargs={"lr":1e-2},
                               batch_size=4, epochs=25, verbose=True, callback=callback)

tensor(0.0381) tensor(0.0317)
tensor(1.4638, grad_fn=<MeanBackward0>) tensor(1.4274, grad_fn=<MeanBackward0>)


NameError: name 'fit' is not defined

In [6]:
theta = torch.zeros(m + 1, m)
theta0 = torch.zeros(m, requires_grad=True)
theta.requires_grad_(True)
opt = torch.optim.LBFGS([theta, theta0], max_iter=500)
l = torch.nn.KLDivLoss()

def closure():
    opt.zero_grad()
    loss = l(torch.nn.LogSoftmax()(X @ theta + theta0), Y / Y.sum(1)[:,None])
    loss.backward()
    return loss

opt.step(closure)

<ipython-input-6-bf5cdb10a48c>:9: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = l(torch.nn.LogSoftmax()(X @ theta + theta0), Y / Y.sum(1)[:,None])
/home/martin/anaconda3/envs/routing_identification/lib/python3.8/site-packages/torch/nn/functional.py:2247: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


tensor(0.0728, grad_fn=<KlDivBackward>)

In [9]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(5.485, 1.8)

ax[0].axhline(loss_fn(torch.nn.Softmax()(Xval @ theta + theta0) * Xval[:,0][:,None], Yval).item(), linestyle='-.', c='black', label='LR')
ax[0].plot(val_losses, c='black', label='COM')
ax[0].axhline(loss(Xval, Yval, alpha_true).item(), linestyle='--', c='black', label='true')
ax[0].legend()
ax[0].set_xlabel("iteration")
ax[0].set_ylabel("validation loss")

ax[1].plot(alpha.detach().numpy(), c='black', label='learned')
ax[1].plot(alpha_true.detach().numpy(), '--', c='black', label='true')
ax[1].set_xlabel("$i$")
ax[1].set_ylabel("$\\theta_i$")
ax[1].legend()

plt.tight_layout()

plt.savefig("figures/resource_allocation.pdf")

NameError: name 'plt' is not defined

NameError: name 'plt' is not defined